In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display, clear_output
import scipy.signal
import requests
import warnings
warnings.filterwarnings('ignore')

# Replace the URL with the URL of the raw content
url = "https://raw.githubusercontent.com/ntua-el17840/Interactive-Digital-Communications/main/test-book/_static/sima.txt"
response = requests.get(url)

# Split the response text by new lines and convert each line to a float
s = np.array([float(line) for line in response.text.splitlines()])

# Sampling frequency
Fs = 8192

# Low-pass filter coefficients dictionary (lp_filters)
lp_filters = {
    'lpass32': scipy.signal.remez(33, [0, 0.1 * Fs, 0.15 * Fs, 0.5 * Fs], [1, 0], Hz=Fs),
    'lpass64': scipy.signal.remez(65, [0, 0.1 * Fs, 0.15 * Fs, 0.5 * Fs], [1, 0], Hz=Fs),
    'lpass128': scipy.signal.remez(129, [0, 0.1 * Fs, 0.15 * Fs, 0.5 * Fs], [1, 0], Hz=Fs),
    'lpass140': scipy.signal.remez(141, [0, 0.1 * Fs, 0.15 * Fs, 0.5 * Fs], [1, 0], Hz=Fs)
}

# Output widget for the plot
plot_output = widgets.Output()

# Function to display the frequency response of the filtered signal
def plot_freq_response(filter_name):
    with plot_output:
        clear_output(wait=True)
        # Apply the selected filter to the original signal
        s_lp_filtered = scipy.signal.convolve(s, lp_filters[filter_name], mode='same') / np.sum(lp_filters[filter_name])
        
        # Calculate FFT of the filtered signal
        freqs = np.fft.rfftfreq(len(s_lp_filtered), 1 / Fs)
        fft_mag = np.abs(np.fft.rfft(s_lp_filtered))
        
        # Convert magnitude to dB scale
        fft_mag_db = 20 * np.log10(fft_mag)
        
        # Plot the frequency response
        plt.figure(figsize=(10, 5))
        plt.plot(freqs, fft_mag_db)
        plt.title(f'Frequency Response of Filtered Signal with {filter_name}')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude (dB)')
        plt.grid(True)
        plt.xlim(0, Fs / 2)
        plt.show()

# Dropdown widget to select the filter
filter_dropdown = widgets.Dropdown(
    options=[(name, name) for name in lp_filters.keys()],
    value='lpass32',
    description='Filter:'
)

# Function to handle dropdown selection change
def on_filter_change(change):
    filter_name = change['new']
    plot_freq_response(filter_name)

filter_dropdown.observe(on_filter_change, names='value')

# VBox layout for alignment and presentation
vbox_layout = widgets.Layout(align_items='center', justify_content='center')
vbox_loader = widgets.HBox([filter_dropdown], layout=vbox_layout)

# Final VBox layout for the filter selection and plot display
vbox_final = widgets.VBox([vbox_loader, plot_output])

# Display the final layout
display(vbox_final)

# Initialize the plot with the default filter
on_filter_change({'new': filter_dropdown.value})
